In [1]:
import numpy as np
import pandas as pd

In [66]:
predictions = ([['__label__9003A', '__label__5911A'], ['__label__4321A', '__label__4321B'], ['__label__8211Z', '__label__8219Z']], [np.array([0.692652 , 0.3208313]), np.array([1.00001   , 0.00256091]), np.array([0.754925  , 0.06372499])])
predictions

([['__label__9003A', '__label__5911A'],
  ['__label__4321A', '__label__4321B'],
  ['__label__8211Z', '__label__8219Z']],
 [array([0.692652 , 0.3208313]),
  array([1.00001   , 0.00256091]),
  array([0.754925  , 0.06372499])])

In [69]:
len(predictions[0])

3

In [48]:
np.diff(predictions[1][0]).astype(float)

array([], dtype=float64)

In [44]:
prob1 = predictions[1][0][0]
ic = np.diff(predictions[1][0])[0] * -1
pred = predictions[0][0][0].replace("__label__", "")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [32]:
x = [[predictions[1][i][0], np.diff(predictions[1][i])[0] * -1, predictions[0][i][0].replace("__label__", "")] for i in range(len(predictions[0][0]))]

In [35]:
df = pd.DataFrame(x, columns=["IC", "Probability", "Prediction"])

In [42]:
df.to_dict()

{'IC': {0: 1.00001, 1: 0.03515484},
 'Probability': {0: 0.9989012700000001, 1: 0.0010444100000000026},
 'Prediction': {0: '6820A', 1: '0149Z'},
 'Code': {0: '6820A', 1: '5813D'},
 'Result': {0: True, 1: False}}

In [5]:
df = pd.read_parquet("../data/data_sirene3.parquet")

In [2]:
df = pd.read_parquet("../data/data_gu.parquet")

In [3]:
df.tail(50).to_csv("../data/exemple.csv")

In [5]:
df

,DATE,APE_SICORE,LIB_SICORE,AUTO,NAT_SICORE,EVT_SICORE,SURF
LIA_NUM,,,,,,,
J00010000461,2021-06-07 08:45:48,5912Z,REALISATION DE DESSINS D ANIMATION,X,None,01P,None
J00010000479,2021-07-28 15:37:51,4321A,ELECTRICIEN ELECTRICITE GENERALE,Z,14,None,None
J00010000818,2021-10-19 19:33:16,8211Z,PRESTATIONS DE SERVICES ASSISTANCE AUX RESSO...,X,99,01P,None
J00010000842,2021-12-02 19:35:11,9609Z,BIEN ETRE ET SOINS ENERGETIQUES DU CORPS ET DE...,X,99,05P,None
J00010001196,2021-12-27 19:32:15,6810Z,MARCHAND DE BIENS CREATION DE LOTISSEMENTS E...,C,99,01M,None
...,...,...,...,...,...,...,...
J00010363844,2022-07-01 11:31:48,9602B,PROTHESISTE ONGULAIRE,X,None,01P,None
J00010363604,2022-07-01 11:31:40,8121Z,PRESTATION DE SERVICE DE MENAGE CHEZ LES PAR...,X,None,01P,None
J00010353084,2022-07-01 10:17:13,6622Z,AGENT GENERAL D ASSURANCE 6622Z,L,None,05P,None


In [6]:
df_gu = df[df.index.str.startswith("J")]

In [11]:
df_gu.to_parquet("../data/data_gu.parquet")

In [35]:
df = pd.read_csv("../data/Jeu_codif_APE_S4_20230606_processed.csv",index_col=[0])
df

,text_description,type_,nature,surface,event,code
0,Nettoyage tout type de locaux,X,NaN,NaN,01P,8121Z
1,SOUDURE,M,99.0,NaN,01P,3320A
2,"Retouche et confection de rideaux, stores bate...",M,4.0,NaN,01P,1392Z
3,"prestations de services, déménagements",M,99.0,NaN,01P,4942Z
4,Livraison uber eats,C,99.0,NaN,01P,5320Z
...,...,...,...,...,...,...
440422,"L'administration et l'exploitation par bail, l...",G,NaN,NaN,01M,6820B
440423,Centre d'appel.,C,99.0,NaN,01M,8220Z
440424,"l'acquisition, l'administration et la gestion,...",G,NaN,NaN,01M,6820B
440425,Production d'électricité photovoltaïque inféri...,I,NaN,NaN,01P,3511Z


In [3]:
username = "codification-ape"
password = "codification-sirene4"

In [90]:
def query_batch_api(username: str, password: str, data: pd.DataFrame):
    url = "http://codification-ape.lab.sspcloud.fr/evaluation"

    # Create the request body as a dictionary from the DataFrame
    request_body = data.to_dict(orient="list")

    response = requests.post(url, json=request_body, auth=(username, password))
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 400:
        print(response.json()["detail"])
    else:
        print("Error occurred while querying the API.")
        return response

In [101]:
def query_batch_api(url, username: str, password: str, data: pd.DataFrame):

    # Create the request body as a dictionary from the DataFrame
    request_body = data.to_dict(orient="list")

    response = requests.post(url, json=request_body, auth=(username, password))
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 400:
        print(response.json()["detail"])
    else:
        print("Error occurred while querying the API.")
        return response

In [5]:
data = df_gu.reset_index()[["LIB_SICORE", "AUTO", "NAT_SICORE", "SURF", "EVT_SICORE", "APE_SICORE"]]
data.columns = ["text_description", "type_", "nature", "surface", "event", "code"]
#data= data.fillna(value="")

In [6]:
data.tail(50).to_csv("../data/exemple_all.csv")

In [5]:
data[:100]

,Unnamed: 0,text_description,type_,nature,surface,event,code
0,0,Nettoyage tout type de locaux,X,NaN,NaN,01P,8121Z
1,1,SOUDURE,M,99.0,NaN,01P,3320A
2,2,"Retouche et confection de rideaux, stores bate...",M,4.0,NaN,01P,1392Z
3,3,"prestations de services, déménagements",M,99.0,NaN,01P,4942Z
4,4,Livraison uber eats,C,99.0,NaN,01P,5320Z
...,...,...,...,...,...,...,...
95,95,"Création vêtements, accessoires et décoration",C,99.0,NaN,01P,7410Z
96,96,Location chambres d'hôtes assurée sur le lieu ...,C,99.0,NaN,01P,5520Z
97,97,réparation cycle,M,99.0,NaN,01P,9529Z
98,98,Hypnose et techniques énergétiques / Vibratoir...,L,NaN,NaN,01P,9609Z


In [11]:
x=query_batch_api("codification-ape","codification-sirene4", df)

InvalidJSONError: Out of range float values are not JSON compliant

In [79]:
pd.DataFrame(x)

,IC,Probability,Prediction,Code,Result
0,0.692652,0.371821,9003A,5912Z,False
1,1.000010,0.997449,4321A,4321A,True
2,0.754925,0.691200,8211Z,8211Z,True
3,0.997120,0.728169,9609Z,9609Z,True
4,0.994455,0.926898,6810Z,6810Z,True
...,...,...,...,...,...
15455,0.993522,0.989313,9602B,9602B,True
15456,0.945005,0.873403,8121Z,8121Z,True
15457,0.999457,0.999447,6622Z,6622Z,True
15458,1.000010,0.999527,6831Z,6831Z,True


In [10]:
import requests
import pandas as pd
from urllib.parse import urlencode
import unidecode

In [196]:
data = pd.read_csv("../data/Jeu_codif_APE_S4_20230606.csv")
data['activ_nat_et'] = data['activ_nat_et'].apply(lambda x: "{:02d}".format(int(x)) if pd.notnull(x) else np.nan)
data = data[["activ_pr_lib_et", "liasse_type", "activ_nat_et", "activ_surf_et", "evenement_type", "apet_finale"]]
data.columns = ["text_description" , "type_", "nature","surface", "event","code"]
data = data.dropna(subset=["text_description"])
data['nature'] = data['nature'].apply(lambda x: "{:02d}".format(int(x)) if pd.notnull(x) else "")
data['surface'] = data['surface'].apply(lambda x: "" if pd.isnull(x) else ("1" if x < 121 else ("2" if 121 <= x <= 400 else ("3" if 401 <= x <= 2500 else "4"))))
#df.to_csv("Jeu_codif_APE_S4_20230606_processed.csv")


In [197]:
idx = get_index_nan_after_prepro(data, "text_description")

In [198]:
data.shape

(430467, 6)

In [204]:
data.iloc[414280:414285]

,text_description,type_,nature,surface,event,code
423828,conseil aux entreprises,X,,,01P,7022Z
423829,conseil en marketing communication,C,99,,01P,7022Z
423830,ape,R,,,01P,6831Z
423831,montage demontage de stands evenementiels,M,14,,01P,4399B
423832,animation musical evenements,X,,,01P,9329Z


In [ ]:
get_index_nan_after_prepro(data, "text_description")

In [207]:
get_index_nan_after_prepro(data.iloc[414280:414285], "text_description")

Index([423830], dtype='int64')

In [199]:
data = data.drop(idx)
data.shape

(430384, 6)

In [192]:
url = "http://0.0.0.0:5000/evaluation"
#query_batch_api(url, "codification-ape","codification-sirene4", data.head(10000))
query_batch_api(url, "toto","toto",data.iloc[414280:41428])

Error occurred while querying the API.


<Response [500]>

In [164]:
pd.unique(data.loc[x.index].text_description)

array(['SANS ACTIVITE', 'Siège Social', 'APE 4932Z', 'Idem siège social',
       'siège social', 'Siège social', 'VOIR OBJET SOCIAL',
       'SIEGE SOCIAL', 'IDEM', 'code NAF : 96.09.19',
       'idem activités principales.', 'xxx', 'sans activité', '.',
       'idem activités principales', 'Voir objet social', '6831Z',
       'Idem activités principales', 'idem siège social', '53.20Z', 'T3P',
       '07', 'V.D.I.', '88.99.19', 'voir objet social', 'code APE 8129B',
       '7021Z', 'Idem', 'idem', '///', 'Cf statuts',
       'Idem activités principales.', 'siege social',
       'Idem activité principale', '6820A', 'V T C', '4339z', '4312A',
       '81.21Z', 'voir objet social annexé', '45.11Z'], dtype=object)

In [206]:
def get_index_nan_after_prepro(dataframe, text_feature):
    df = dataframe.copy()
    df[text_feature] = df[text_feature].map(unidecode.unidecode)

    # On passe tout en minuscule
    df[text_feature] = df[text_feature].str.lower()

    # define replacement patterns
    replacements = {
        # Libellé vide de sens fournit par Christine (DOIT ETRE FAIT EN AMONT DU MODELE EN JAVA)
        r"\bidem\b|\bvoir ci dessous\b|\[vide\]|\bundefined\b|\bpas d objet\b|\(voir ci dessus\)|\(voir extrait siege social\\/etablissement principal\)|\bcf activite principale\b|\bcf activite principale et objet\b|\bcf activites de l entreprise\b|\bcf activites principales de l entreprise\b|\bcf actvites principales\b|\bcf k bis\b|\bcf le principales activites de l  entreprise\b|\bcf le sprincipale activites de l  entreprise\b|\bcf le sprincipales activites de l  entreprise\b|\bcf les activites principales de l  entreprise\b|\bcf les ppales activites de l  entreprise\b|\bcf les ppales activites de la ste\b|\bcf les principale activites de l  entreprise\b|\bcf les principales activites\b|\bcf les principales activites de l  entreprise\b|\bcf les principales activites de l  entreprises\b|\bcf les principales activites ppales de l  entreprise\b|\bcf les principales activtes de l  entreprise\b|\bcf les principales acttivites de l  entreprise\b|\bcf les prinipales activites de l  entreprise\b|\bcf lesprincipales activites de l  entreprise\b|\bcf objet\b|\bcf obs\b|\bcf principales activite de l  entreprise\b|\bcf principales activites de l  entreprise\b|cf rubrique \"principales activites de l entreprise\" idem|cf rubrique n2 ci dessus \(743b\)|\bcf supra\b|\bcf ci  dessus\b|\bcommerce de detail, idem case 2\b|\bextension a: voir ci dessus\b|\bid\b|\bid principales activites\b|\bid principales activites de l  entreprise\b|\bidem ci dessus\b|idem \( voir principales activites\)|\bidem  dessus\b|\bidem 1ere page\b|\bidem a principales activites de l  entreprise\b|\bidem activiet eprincipale\b|\bidem activite\b|\bidem activite 1ere page\b|\bidem activite ci  dessus\b|\bidem activite de l  entreprise\b|\bidem activite enoncee ci  dessus\b|\bidem activite entreprise\b|\bidem activite generales\b|\bidem activite premiere page\b|\bidem activite principale\b|\bidem activite princippale\b|\bidem activite prinicpale\b|\bidem activite sur 1ere page\b|\bidem activites ci dessus\b|\bidem activites declarees au siege et principal\b|\bidem activites enoncees ci dessus\b|\bidem activites entreprise\b|\bidem activites principales\b|\bidem activites principales de l entreprise\b|\bidem activites siege\b|\bidem activte principale\b|\bidem activtie 1ere page\b|\bidem au siege\b|\bidem au siege social\b|\bidem aux principales actiivtes\b|\bidem aux principales activites\b|\bidem case 13\b|\bidem ci dessous\b|\bidem ci dessus enoncee\b|\bidem cidessus\b|\bidem objet\b|\bidem premiere page\b|\bidem pricincipales activites de l entreprise\b|\bidem pricipales activites\b|\bidem principale activite\b|\bidem principales activite de l entreprise\b|\bidem principales activite de l entreprises\b|\bidem principales activite l entreprise\b|\bidem principales activites\b|\bidem principales activites citees ci dessus\b|\bidem principales activites de l entreprises\b|idem principales activites de l entreprise\(objet\)|\bidem principales activites et objet social\b|\bidem principales activitse de l entreprise\b|\bidem que celle decrite plus haut\b|\bidem que ci dessus\b|\bidem que l activite decrite plus haut\b|\bidem que les activites principales\b|\bidem que les activites principales ci dessus\b|\bidem que les activitges principales\b|\bidem que les principales activites\b|\bidem que les principales activites de l entreprise\b|\bidem que pour le siege\b|\bidem rubrique principales activites de l entreprise\b|\bidem siege\b|idem siege \+ voir observation|\bidem siege et ets principal\b|\bidem siege social\b|idem siege, \(\+ articles americains\)|\bidem societe\b|\bidem voir activite principale\b|\bidem voir ci dessus\b|\bidentique a l objet social indique en case 2 de l imprime m2\b|\bidm ci dessus\b|\bnon indiquee\b|\bnon precise\b|\bnon precisee\b|\bnon precisees\b|\bvoir 1ere page\b|\bvoir activite ci dessus\b|\bvoir activite principale\b|\bvoir activite principale ci dessus\b|\bvoir activites principales\b|\bvoir cidessus\b|\bvoir idem ci dessus\b|\bvoir objet social\b|\bvoir page 1\b|\bvoir page precedente\b|\bvoir plus haut\b|\bvoir princiale activite\b|\bvoir princiales activites\b|\bvoir princiapales activites\b|\bvoir princiaples activites\b|\bvoir principale activite\b|\bvoir principales activites\b|\bvoir principales activites de l entreprise\b|\bvoir principales actvites\b|\bvoir principalesactivites\b|\bvoir principles activites\b|\bvoir rubrique principales activites de l entreprise\b|\bvoir sur la 1ere page\b|\bvoir dessus\b|voir: \"principales activite de l entreprise\"|voir: \"principales activites de l entreprises\"|voir: \"principales activites de l entrprise\"|voir: \"principales activites en entreprise\"|\bconforme au kbis\b|\bsans changement\b|\bsans activite\b|\bsans acitivite\b|\bactivite inchangee\b|\bactivites inchangees\b|\bsiege social\b|\ba definir\b|\ba preciser\b|\bci dessus\b|\bci desus\b|\bci desssus\b|\bvoir activit principale\b|\bidem extrait kbis\b|\bn a plus a etre mentionne sur l extrait decret\b|\bcf statuts\b|\bactivite principale case\b|\bactivites principales case\b|\bactivite principale\b|\bactivites principales\b|\bvoir case\b|\baucun changement\b|\bsans modification\b|\bactivite non modifiee\b|\bactivite identique\b|\bpas de changement\b|\bcode\b|\bape\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bxx\b|\bxxx\b|\binconnue\b|\binconnu\b|\bvoir\b|\bannexe\b|\bmo\b|\biem\b|\binchanges\b|\bactivite demeure\b|\bactivite inchangée\b|\bcase precedente\b|\bidem cadre precedent\b|\bactivite demeure\b|\bactivite inchangée\b|\bnon renseignee\b|\bneant\b|\bnon renseigne\b": " ",
        # supprime hyphen pour les mots comme e-commerce
        r"e-": "e",
        # accole le e pour les mots comme e-commerce
        r"\be\s": " e",
        # On supprime toutes les ponctuations
        r"[^\w\s]": " ",
        # On supprime certains mots sans sens (DOIT ETRE FAIT DANS LE PREPROCESSING EN JAVA)
        r"\bcode\b|\bcadre\b|\bape\b|\bape[a-z]{1}\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bidem\b|\bxx\b|\bxxx\b|\baa\b|\baaa\b|\bidem cadre precedent\b|\bidem case\b|\binchanges\b|\bmo\b|\biem\b|\bci dessus\b|\bet\b": "",
        # On supprime tous les chiffres
        r"[\d+]": " ",
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].str.replace(
            pattern, replacement, regex=True
        )

    # On supprime les mots d'une seule lettre
    df[text_feature] = df[text_feature].apply(
        lambda x: " ".join([w for w in x.split() if len(w) > 1])
    )

    # define replacement patterns
    replacements = {
        # On supprime les espaces multiples
        r"\s\s+": " ",
        # On strip les libellés
        r"^\s+|\s+$": "",
        # On remplace les empty string par des NaN
        r"^\s*$": np.nan,
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].replace(
            pattern, replacement, regex=True
        )

    return df[pd.isnull(df).any(axis=1)].index